## Movies on Streaming (OTT) Platforms and Exploratory Analysis

here we take a look at a cool movies dataset where we take a look at movies on:

- Netflix
- Amazon Prime
- Hulu
- Disney +

Note: OTT = Over-the-top

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv("https://github.com/rajeevratan84/datascienceforbusiness/blob/master/MoviesOnStreamingPlatforms_updated.csv?raw=true")
df

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16739,16740,The Ghosts of Buxley Hall,1980,NaN,6.2,NaN,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.0
16740,16740,16741,The Poof Point,2001,7+,4.7,NaN,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.0
16741,16741,16742,Sharks of Lost Island,2013,NaN,5.7,NaN,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,NaN
16742,16742,16743,Man Among Cheetahs,2017,NaN,6.6,NaN,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,NaN


# Let's explore and visualize the movie genres

In [3]:
# add genres as categories and then merge the dataframes
genres = df['Genres'].str.get_dummies(',')
df = pd.concat([df, genres], axis=1, sort=False)

# remove the unnamed axis and fill with NaNs
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.fillna(np.nan, inplace=True)
df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
genres

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
16740,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
16741,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16742,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Check for missing values and visualize their occurances

In [4]:
# missing value totals
total = df.isnull().sum().sort_values(ascending=False)
total.head()

Rotten Tomatoes    11586
Age                 9390
Directors            726
Language             599
Runtime              592
dtype: int64

In [5]:
# Now let's put it as a percentage
percent = ((df.isnull().sum()/df.isnull().count())*100).sort_values(ascending=False)
percent

Rotten Tomatoes    69.194935
Age                56.079790
Directors           4.335882
Language            3.577401
Runtime             3.535595
IMDb                3.410177
Country             2.597946
Genres              1.642379
Western             0.000000
Animation           0.000000
Adventure           0.000000
Action              0.000000
Disney+             0.000000
Type                0.000000
Comedy              0.000000
Prime Video         0.000000
Hulu                0.000000
Netflix             0.000000
Year                0.000000
Title               0.000000
Biography           0.000000
Crime               0.000000
War                 0.000000
Mystery             0.000000
Thriller            0.000000
Talk-Show           0.000000
Sport               0.000000
Short               0.000000
Sci-Fi              0.000000
Romance             0.000000
Reality-TV          0.000000
News                0.000000
Musical             0.000000
Documentary         0.000000
Music         

In [6]:
# let's put total and percentage missing in one dataframe 
missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])[:8]
missing.head()

,Total,Percent
Rotten Tomatoes,11586,69.194935
Age,9390,56.079790
Directors,726,4.335882
Language,599,3.577401
Runtime,592,3.535595


In [7]:
# creating a nice viz
fig = px.bar(missing, x=missing.index,
              y = 'Total',
             title= 'Missing values in each column',
             hover_data = ['Percent'],
             labels = {'index':'Column'})
fig.update_traces(marker_color='#fd79a8')
fig.show()

# 2. Visualizations - where are the best movies found? Netflix, Disney +, Prime or Hulu? 

In [8]:
# platforms
netflix = len(df[df['Netflix']==1])
hulu = len(df[df['Hulu']==1])
prime = len(df[df['Prime Video']==1])
disney = len(df[df['Disney+']==1])
Platform = ['Netflix', 'Hulu', 'Prime Video', 'Disney+']
Count = [netflix, hulu, prime, disney]

fig = px.pie(names = Platform,
            values=Count,
             title='Movie Count of Different Platforms',
             color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Platforms with IMDB 8.5 + Movies

In [9]:
# Let's create new dataframes for each platform 
netflix_movies = df.loc[df['Netflix'] == 1].drop(['Hulu', 'Prime Video', 'Disney+', 'Type'],axis=1)
hulu_movies = df.loc[df['Hulu'] == 1].drop(['Netflix', 'Prime Video', 'Disney+', 'Type'],axis=1)
prime_video_movies = df.loc[df['Prime Video'] == 1].drop(['Hulu', 'Netflix', 'Disney+', 'Type'],axis=1)
disney_movies = df.loc[df['Disney+'] == 1].drop(['Hulu', 'Prime Video', 'Netflix', 'Type'],axis=1)

# Define our minimum ratings 
min_rating = 8.5

# Get number of movies that have IMDB scores above our minimum rating 
count_imdb = [len(netflix_movies[netflix_movies['IMDb']>min_rating]),
              len(hulu_movies[hulu_movies['IMDb']>min_rating]),
              len(prime_video_movies[prime_video_movies['IMDb']>min_rating]),
              len(disney_movies[disney_movies['IMDb']>min_rating])]

# List of platform names
platform = ['Netflix', 'Hulu', 'Prime Video', 'Disney+']

# Create our dataset we'll visualize below
top_rated = pd.DataFrame({'Platforms':platform, 'Count': count_imdb})

# Generate our bar chart
fig = px.bar(top_rated,
             x = 'Platforms',
             y = 'Count',
             color = 'Count',
             color_continuous_scale = 'Rainbow',
             title='IMDb 8.5+ Movies on different Platforms')

fig.show()

In [ ]:
top_rated

,Platforms,Count
0,Netflix,16
1,Hulu,4
2,Prime Video,80
3,Disney+,2


## Now let's see the top movies on each platform

In [10]:
n = netflix_movies.sort_values('IMDb', ascending=False).head(10)
h = hulu_movies.sort_values('IMDb', ascending=False).head(10)
p = prime_video_movies.sort_values('IMDb', ascending=False).head(10)
d = disney_movies.sort_values('IMDb', ascending=False).head(10)

fig = make_subplots(rows=4, cols=1, subplot_titles=("Top 10 Movies on Netflix", "Top 10 Movies on Hulu",
                                                    "Top 10 Movies on Prime Video","Top 10 Movies on Disney+"))

fig.add_trace(go.Bar(y=n['Title'], x=n['IMDb'], orientation='h', marker=dict(color=n['IMDb'],coloraxis='coloraxis')), row=1, col=1)
fig.add_trace(go.Bar(y=h['Title'], x=h['IMDb'], orientation='h', marker=dict(color=n['IMDb'],coloraxis='coloraxis')), row=2, col=1)
fig.add_trace(go.Bar(y=p['Title'], x=p['IMDb'], orientation='h', marker=dict(color=n['IMDb'],coloraxis='coloraxis')), row=3, col=1)
fig.add_trace(go.Bar(y=d['Title'], x=d['IMDb'], orientation='h', marker=dict(color=n['IMDb'],coloraxis='coloraxis')), row=4, col=1)

fig.update_layout(height=1300, width=1000, title_text="Top Movies on Different Platforms based on IMDb Rating",
                  coloraxis=dict(colorscale='Rainbow'), showlegend=False)

fig.show()

## Movies per year

In [11]:
# Year
year_count = df.groupby('Year')['Title'].count()
year_movie = df.groupby('Year')[['Netflix', 'Hulu', 'Prime Video', 'Disney+']].sum()
year_data = pd.concat([year_count,year_movie], axis=1).reset_index().rename(columns={'Title': 'Movie Count'})
year_data.head()

,Year,Movie Count,Netflix,Hulu,Prime Video,Disney+
0,1902,1,0,0,1,0
1,1912,1,0,0,1,0
2,1913,2,0,0,2,0
3,1915,4,0,0,4,0
4,1916,1,0,0,1,0


In [12]:
# Generate plot
fig = px.bar(year_data,
             x = 'Year',
             y = 'Movie Count',
             hover_data = ['Netflix', 'Hulu', 'Prime Video', 'Disney+'],
             color = 'Movie Count',
             color_continuous_scale='Rainbow',
             title='Movie Count by Year')

fig.show()

# 3.0 Best Movies Per Year, Directors & Genres

In [ ]:
# We use .first() to sort on eraliest date first
df.sort_values('IMDb', ascending=False).groupby('Year').first().reset_index().head()

,Year,ID,Title,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1902,4468,A Trip to the Moon,all,8.2,100%,0,0,1,0,0,Georges Méliès,"Short,Action,Adventure,Comedy,Fantasy,Sci-Fi",France,"None,French",13.0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1,1912,8796,From the Manger to the Cross,7+,5.7,None,0,0,1,0,0,Sidney Olcott,"Biography,Drama",United States,English,60.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1913,9918,Fatty Joins the Force,None,5.3,None,0,0,1,0,0,George Nichols,"Comedy,Short",United States,English,12.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1915,4913,The Birth of a Nation,7+,6.4,93%,0,0,1,0,0,D.W. Griffith,"Drama,History,War",United States,None,195.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1916,4620,Intolerance: Love's Struggle Throughout the Ages,None,7.8,97%,0,0,1,0,0,D.W. Griffith,"Drama,History",United States,English,163.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
 best_movie_year = df.sort_values('IMDb', ascending=False).groupby('Year').first().reset_index()

 fig = px.scatter(best_movie_year,
                  x = 'Year',
                  y = 'IMDb',
                  hover_data = ['Title', 'Runtime', 'Genres', 'Language'],
                  color_continuous_scale = 'Rainbow',
                  color = 'IMDb',
                  size = 'IMDb',
                  title = 'Best Movie Each Year According to IMDb Rating')
 
 fig.show()

In [ ]:
df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
year_runtime = df.groupby('Year')['Runtime'].mean().reset_index()
year_runtime

,Year,Runtime
0,1902,13.000000
1,1912,60.000000
2,1913,10.000000
3,1915,73.500000
4,1916,163.000000
...,...,...
104,2016,93.992140
105,2017,94.460961
106,2018,94.635678
107,2019,93.410413


In [ ]:
fig = px.scatter(year_runtime, x='Year', y='Runtime')
fig.show()

## Who are Best Directors? 

In [ ]:
dir_count = df.groupby('Directors')['Title'].count()
dir_movie = df.groupby('Directors')[['Netflix', 'Hulu', 'Prime Video', 'Disney+']].sum()
dir_rating = df.groupby('Directors')['IMDb'].mean()
dir_data = pd.concat([dir_count,dir_movie,dir_rating], axis=1).reset_index().rename(columns={'Title':'Movie Count',
                                                                                            'IMDb': 'Average Rating'})

dir_count_data = dir_data.sort_values("Movie Count", ascending=False).head(10)
dir_count_data.head()

,Directors,Movie Count,Netflix,Hulu,Prime Video,Disney+,Average Rating
4686,Jay Chapman,36,12,0,29,0,6.472222
5618,Joseph Kane,30,0,0,30,0,6.240000
1771,Cheh Chang,26,2,0,25,0,6.757692
5048,Jim Wynorski,22,0,0,22,0,3.681818
9462,Sam Newfield,22,0,0,22,0,5.468182


In [ ]:
dir_rating_data = dir_data.sort_values('Average Rating', ascending=False).head(10)

fig = px.bar(dir_rating_data,
             x = 'Directors',
             y = 'Average Rating',
             hover_data=['Movie Count', 'Netflix', 'Hulu', 'Prime Video', 'Disney+'],
             color = 'Average Rating',
             color_continuous_scale = 'Rainbow',
             title = 'Top 10 Directors with Movies Having Highest IMDb Rating')

fig.show()

## Now let's explore the movie genres

In [ ]:
gen_count = df.groupby('Genres')['Title'].count()
gen_movie = df.groupby('Genres')[['Netflix', 'Hulu', 'Prime Video', 'Disney+']].sum()
gen_data = pd.concat([gen_count, gen_movie], axis=1).reset_index().rename(columns={'Title':'Movie Count'})
# sort by top 10
gen_data = gen_data.sort_values('Movie Count', ascending=False)[:10]

fig = px.bar(gen_data,
             x='Genres',
             y='Movie Count',
             hover_data=['Netflix', 'Hulu', 'Prime Video', 'Disney+'],
             color = 'Movie Count',
             color_continuous_scale = 'Rainbow',
             title="Top 10 Genres Movie Count")

fig.show()

## Which Genres received the best ratings? 

In [ ]:
top_ten_genres = list(gen_data['Genres'].unique())

In [ ]:
gen_rating = df.groupby('Genres')['IMDb'].mean().reset_index()
gen_rating = gen_rating[gen_rating['Genres'].isin(top_ten_genres)]
gen_rating = gen_rating.sort_values('IMDb', ascending=False)

fig = px.bar(gen_rating,
             x='Genres',
             y = 'IMDb',
             color = 'IMDb',
             color_continuous_scale = 'Viridis',
             title = 'Average Review of Each Genre')

fig.show()

In [ ]:
import plotly.express as px

fig = px.colors.sequential.swatches()
fig.show()